In [9]:
import pandas as pd
from datetime import datetime, timedelta
import os
import time

# Assuming fetch_newspaper_data is imported from the KBDownloader module
from KBDownloader import fetch_newspaper_data

# Load the Excel file
excel_filepath = '/Users/brandonfarnsworth/Library/Mobile Documents/com~apple~CloudDocs/Post-Phd/Quantitative Work/Software/Oldtimey_touringbot/test_lokalerna.xlsx'
df = pd.read_excel(excel_filepath)
#Get today's date
today_date = datetime.today().strftime('%Y-%m-%d')

# Initialize start date for data fetching
start_year = 1908  # Adjust to your starting year
years_to_crawl = 1  # Number of years to crawl

name_of_newspaper = 'Svenska Dagbladet'

# Main loop over the specified year range
for year in range(start_year, start_year + years_to_crawl):
    for half in range(2):  # Loop for each half of the year
        if half == 0:
            from_date = datetime(year, 1, 1)  # Start of the year
            to_date = datetime(year, 6, 30)  # End of June
        else:
            from_date = datetime(year, 7, 1)  # Start of July
            to_date = datetime(year, 12, 31)  # End of the year

        for index, row in df.iterrows():
            query = row['Lokal']
            safe_query = "".join([c if c.isalnum() else "_" for c in query])
            output_dir = f'extracted_data_{safe_query}_{today_date}'
            os.makedirs(output_dir, exist_ok=True)
            output_filepath = os.path.join(output_dir, f'extracted_data_{safe_query}_{today_date}')

            result = fetch_newspaper_data(
                query=query,
                from_date=from_date.strftime('%Y-%m-%d'),  
                to_date=to_date.strftime('%Y-%m-%d'),   
                # - newspaper (str): Valid options are: 'Dagens nyheter', 'Svenska Dagbladet', 'Aftonbladet', 'Dagligt Allehanda'. 
                newspaper=name_of_newspaper,
                prompt_filepath='oldtimey_touringbot_prompt_for_deployment.txt',
                output_filepath=output_filepath
            )
            
            if result.get('success') and result.get('message') == 'Data processing and export completed successfully.':
                print(f"Processed query '{query}' successfully.")
            else:
                print(f"Failed to process query '{query}': {result.get('message')}")

        # Wait for a minute after each half-year data collection
        print(f"waiting so KB does not get mad. Currently at{from_date} to {to_date}")
        time.sleep(10)

print("All queries processed for all specified years.")

Processed query 'La Croix salong' successfully.
Processed query 'Norra paviljongen i Trädgårdsföreningens lokal' successfully.
Failed to process query 'Wallmans lokal (Mäster Samuels gränd 11)': No data to export. The list of data frames is empty.
Processed query 'Kungliga operan/Kungliga teatern' successfully.
Failed to process query 'F.d. Kirsteinska huset (vid Clara) [Hotel W6 resp. Hotel Continental]': {"detail":"Malformed query"}
Processed query 'La Croix mindre salong' successfully.
Processed query 'Södra teatern i Stadshuset' successfully.
waiting so KB doesn't get mad
Processed query 'La Croix salong' successfully.
Failed to process query 'Norra paviljongen i Trädgårdsföreningens lokal': No data to export. The list of data frames is empty.
Failed to process query 'Wallmans lokal (Mäster Samuels gränd 11)': No data to export. The list of data frames is empty.
Processed query 'Kungliga operan/Kungliga teatern' successfully.
Failed to process query 'F.d. Kirsteinska huset (vid Cla

# Step 2: Clean up: Data created in step 1 is concatenated into jsonl file, folders and XLSX files deleted 

In [16]:
import os
import glob


# Define the path where the JSONL files are stored and where to save the final concatenated JSONL file
final_jsonl_filename = f'final_data_{today_date}_{name_of_newspaper}.jsonl'

# Define the base directory to start the search and the path for the output JSONL file
base_directory = '/Users/brandonfarnsworth/Library/Mobile Documents/com~apple~CloudDocs/Post-Phd/Quantitative Work/Software/Oldtimey_touringbot/extracted_data29.05.'
final_jsonl_filename = f'final_data_{today_date}_{name_of_newspaper}.jsonl'
final_jsonl_filepath = os.path.join('/Users/brandonfarnsworth/Library/Mobile Documents/com~apple~CloudDocs/Post-Phd/Quantitative Work/Software/Oldtimey_touringbot/', final_jsonl_filename)  # Adjust the output file path as needed

print(f"Looking for JSONL files in {base_directory}")
print(f"Final concatenated file will be saved as {final_jsonl_filepath}")

# Open the output file once and write to it as we find JSONL files
with open(final_jsonl_filepath, 'a') as f_out:
    # Walk through the directory structure
    for root, dirs, files in os.walk(base_directory):
        print(f"Checking directory: {root}")
        # Filter and process only JSONL files
        for file in files:
            if file.lower().endswith('.jsonl'):  # This makes the check case-insensitive
                full_path = os.path.join(root, file)
                print(f"Found JSONL file: {full_path}")
                with open(full_path, 'r') as f_in:
                    f_out.write(f_in.read())
                print(f"Added contents of {file} to {final_jsonl_filepath}")
            else:
                print(f"Ignored file: {file}")

print("All JSONL files have been successfully concatenated.")



Looking for JSONL files in /Users/brandonfarnsworth/Library/Mobile Documents/com~apple~CloudDocs/Post-Phd/Quantitative Work/Software/Oldtimey_touringbot/extracted_data29.05.
Final concatenated file will be saved as /Users/brandonfarnsworth/Library/Mobile Documents/com~apple~CloudDocs/Post-Phd/Quantitative Work/Software/Oldtimey_touringbot/final_data_2024-05-29_test.jsonl
Checking directory: /Users/brandonfarnsworth/Library/Mobile Documents/com~apple~CloudDocs/Post-Phd/Quantitative Work/Software/Oldtimey_touringbot/extracted_data29.05.
Ignored file: .DS_Store
Checking directory: /Users/brandonfarnsworth/Library/Mobile Documents/com~apple~CloudDocs/Post-Phd/Quantitative Work/Software/Oldtimey_touringbot/extracted_data29.05./extracted_data_Södra_teatern_i_Stadshuset_2024-05-29
Found JSONL file: /Users/brandonfarnsworth/Library/Mobile Documents/com~apple~CloudDocs/Post-Phd/Quantitative Work/Software/Oldtimey_touringbot/extracted_data29.05./extracted_data_Södra_teatern_i_Stadshuset_2024-05-